# Project File - APS360 Team 25
Divided into the following section: 
# 
1) Library imports
2) Data imports
3) Model architecture definition
4) Training function definition
5) Model training
6) Model testing

## Library imports 
(Place all library imports here)

In [1]:
#KP - I just added the main ones from the labs.
import torch
import numpy as np

#import torchvision
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

import time # Tracking model training time.

#for Data importing
import mido
from mido import MidiFile, Message, MidiTrack, MetaMessage
import os
import random

In [2]:
#Set working directory if required:
os.chdir('D:\engsci\year 3\CLASS\APS360\Project') #Sets current working directory!

## Data imports
#### MIDI reading functions

In [3]:
def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks:
                trackCount += 1
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[1]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)
            
def PrintMetaMessages(mid):             #print fmeta messages
    track = mid.tracks[0]
    for i,msg in enumerate(track):
        print(msg)

def cleanupMessages(mid):              #removes non-note messages by force
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2

#### MIDI to Numpy code

In [4]:

def Midi2NumpyNoSustain(mid):                                #converts to numpy array removing non-note messages
    track = mid.tracks[1]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        else:
            time += msg.time                        #adjust time when removing other messages
    return notes


def NumpyGetSustain(note):
    notes = np.copy(note)
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]
    time = 0
    for i, msg in enumerate(notes):
        if msg[1] > 0:
            notes[i,2] += time
            time = 0
        else:
            time += msg[2]                        #adjust time
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes

def NumpyNormalize(note, oneHot=False):                         #normalize all values to 0-1
    notes = np.copy(note)
    
    if oneHot:
        notes[:,12] /= 11
        notes[:,13] /= 128
        notes[:,14] /= 40000
        notes[:,15] /= 40000
    else:
        notes[:,0] /= 128
        notes[:,1] /= 128
        notes[:,2] /= 40000
        notes[:,3] /= 40000       
    return notes

def NumpyOneHot(note):
    notes = np.copy(note)
    oneHot = np.zeros([len(notes),16])
    oneHot[:, 13:] = notes[:, 1:]
    names = notes[:,0]
    namesOct = names%12
    oneHot[:,12] = (names-(namesOct))/12
    
    for i, name in enumerate(namesOct):
        oneHot[i,name.astype(int)] = 1
    
    return oneHot

def Midi2Numpy(path, oneHot=False): # full midi to numpy conversion
    mid = MidiFile(path)
    notes = Midi2NumpyNoSustain(mid)
    cleanNotes = NumpyGetSustain(notes)
    
    if oneHot:
        cleanNotes = NumpyOneHot(cleanNotes)
    
    normNotes = NumpyNormalize(cleanNotes, oneHot=oneHot)
    return normNotes

#### Numpy to MIDI code

In [5]:

def NumpyDenormalize(note): # interpret all values from 0-1 to normal values
    notes = np.copy(note)    
    if notes.shape[1] == 16: # if encode as one-hot
        notes[:,12] *= 11
        notes[:,13] *= 128
        notes[:,14] *= 40000
        notes[:,15] *= 40000
        
        notes = NumpyEncode(notes) #encode back as original 4-variable format
    else:
        notes[:,0] *= 128
        notes[:,1] *= 128
        notes[:,2] *= 40000
        notes[:,3] *= 40000       
    return notes.astype(int)

def NumpyEncode(note): # convert back from one-hot encoding
    notes = np.copy(note)
    encoded = np.zeros([len(notes),4])
    encoded[:, 1:] = notes[:, 13:]
    encoded[:, 0] = notes[:,12]*12
    
    for i in range(len(notes)):
        encoded[i,0] += np.argmax(notes[i,:12])
    
    return encoded

def NumpySequence(notes): # put all notes into a "timeline" i.e.: time values of [10, 20, 10, 30] become [10, 30, 40, 70]
    sequenced = np.copy(notes)                      # this allows us to easily add vel=0 notes in any order since we can later sort them by time
    for i, msg in enumerate(sequenced):
        if i > 0:
            sequenced[i,2] += sequenced[i-1,2]
    return sequenced

def NumpyAddOffNotes(sequenced): # add vel=0 notes from sustain into sequenced timeline
    withOff = np.copy(sequenced)
    for msg in sequenced:
        offNote = np.array([[msg[0], 0, msg[2] + msg[3], 0]])
        withOff = np.append(withOff, offNote, axis=0)
    #withOff = np.sort(withOff,axis=0)
    withOff = withOff[withOff[:,2].argsort()] # sort by time
    return withOff

def NumpyUnsequence(notes): # revert time value to "time since last message"
    unsequenced = np.copy(notes)
    for i, msg in reversed(list(enumerate(unsequenced))):
        unsequenced[i,3] = 0
        if i > 0:
            unsequenced[i,2] -= unsequenced[i-1,2]
    return unsequenced

def Numpy2MidiDirect(array):    #make MIDI object from numpy
    #Start with initializing a new Mido Track:
    mid = MidiFile()
    track0 = MidiTrack()
    track1 = MidiTrack()
    
    track0.append(MetaMessage('set_tempo', tempo=500000, time=0)) #MetaMessages not necessary but are present in used files
    track0.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
    track0.append(MetaMessage('end_of_track', time=1))
    
    track1.append(Message('program_change', channel=0, program=0, time=0))
    
    for i,note in enumerate(array):         # Get the index and the note. Array must be int array
        j = 1
        track1.append(Message('note_on',note = array[i,0], velocity = array[i,1],time = array[i,2])) # Add the note to the track.

    mid.tracks.append(track0)
    mid.tracks.append(track1)
    return mid

def Numpy2Midi(notes, name): # full numpy to midi conversion, saving result to [name].midi
    denorm = NumpyDenormalize(notes)
    seq = NumpySequence(denorm)
    off = NumpyAddOffNotes(seq)
    unseq = NumpyUnsequence(off)
    mid = Numpy2MidiDirect(unseq)
    mid.save(name + ".midi")

#### Generating tensor dataset from CSVs

In [6]:
def Numpy2Dataset(notes,num=100,skip=10): # make list of sumpy arrays #Playing with window sized (num)
    samples = []
    i = 0
    while i+num <= len(notes):
        samples.append(notes[i:i+num])
        i += skip
    return samples

def SampleAllNumpy(dataPath): # generate samples from all saved CSVs
    allSamples = []

    for i,f in enumerate(os.listdir(dataPath)):
        
        
        if i % 1280 == 0:
            print(i)
            notes = np.genfromtxt(dataPath+f, delimiter=',') #Moved down... 
            allSamples += Numpy2Dataset(notes) #Moved down into if statement...
    
    return allSamples

def SaveSamplesTensor(samples, outputPath): # save tensor
    tens = torch.Tensor(samples)
    torch.save(samples, outputPath+"Notes_Dataset_test_onehot.pt")
    return tens   

def SaveAllSamples(dataPath, outputPath): # save dataset tensor
    samples = SampleAllNumpy(dataPath)
    SaveSamplesTensor(samples, outputPath)
    print('done')

#### Bulk data conversion code - COMMENT OUT IF NOT IN USE!!!

In [7]:
# SaveAllSamples("D:/engsci/year 3/CLASS/APS360/Project/data/numpy_files/","D:/engsci/year 3/CLASS/APS360/Project/data/") #save all into tensor

## Baseline Model Code
#### getting available notes

In [8]:
def GetAllNotesMajor(root):# Get all used notes in major scale of root=root
    notes = []
    intervals = [2,2,1,2,2,2,1]
    
    while root > 24: #bring down to lowest used octave
        root -= 12
    
    n = root
    notes.append(n)
    while n < 84: #up to higherst used note
        for i in intervals:
            n += i
            notes.append(n)   
    return notes    


def GetRangeMajor(notes, low, high): # Get all notes within range
    lowIndex = notes.index(low)
    highIndex = notes.index(high)
    
    return notes[lowIndex:highIndex+1]   

#### Piece Class
##### represents whole output from all 4 voices

In [9]:
class Piece: # Entire baseline model compostion - composed of 4 voices soprano, alto, tenor, bass (SATB)
    def __init__(self, barNum=16, root=60):# 16 bars in C major
        self.root = root # root note
        self.allNotes = GetAllNotesMajor(self.root) # all notes on major scale
        self.barNum = barNum # number of bars
        
        self.soprano = Voice(self.allNotes,60,84,speed=8) # SATB
        self.alto = Voice(self.allNotes,48,72)
        self.tenor = Voice(self.allNotes,36,60)
        self.bass = Voice(self.allNotes,24,48)
          
        self.notes = np.empty([0,4]) #notes output
        
        self.pieceChords = [] # chords
        
        self.chords = np.array([ # common classical C major chords
            [ 0,  4,  7,  0],# I
            [ 2,  5,  9,  2],# ii
            [ 4,  7, 11,  4],# iii
            [ 5,  9, 0,  5],# IV
            [ 7, 11, 2,  7],# V
            [ 9, 0, 4,  9],# vi
            [11, 2, 5, 11],# vii dim
            [ 2,  5,  9, 0],# ii7
            [ 5,  9, 0, 4],# IVmaj7
            [ 7, 11, 2, 5],# V7
            [11, 2, 5, 9]])# vii7 half-dim
        
    def GenerateSoprano(self): # Generate soprano line
        self.soprano.GenerateLine(self.soprano.speed*self.barNum)
        
    def GenerateAlto(self): # Generate alto line from chords
        self.alto.GenerateChordLine(self.pieceChords)
        
    def GenerateTenor(self): # see alto
        self.tenor.GenerateChordLine(self.pieceChords)
        
    def GenerateBass(self): # see alto
        self.bass.GenerateChordLine(self.pieceChords)
        
        
    
    def ChooseChord(self, sopNote): # Choose a fitting chord for soprano note
        while sopNote >= 12:
            sopNote -= 12
        
        goodChords = np.empty([0,4])
        
        for chord in self.chords:
            if (chord==sopNote).sum() > 0:
                goodChords = np.append(goodChords,[chord],axis=0)
        
        chosenChord = goodChords[random.randint(0,len(goodChords)-1)]
        chosenChord = np.sort(np.unique(chosenChord))
        
        i = 12
        chordNotes = chosenChord
        while i < 120:
            chordNotes = np.append(chordNotes, chosenChord+i)
            i += 12
        
        return(chordNotes)
    
    def GetChords(self): # select all chords in piece
        for i, note in enumerate(self.soprano.notes):
            if i % 2 == 0:
                sopNote = note[0]
                chord = self.ChooseChord(sopNote)
                self.pieceChords.append(chord)
                
    def GenerateLines(self): # Generate all SATB lines and joins them - entire baseline model
        self.GenerateSoprano()
        self.GetChords()
        self.GenerateAlto()
        self.GenerateTenor()
        self.GenerateBass()
        self.joinLines()
        
        self.notes = self.notes.astype(int)
        self.OffsetTime(20)
        
        return self.notes
        
    def InsertLine(self, starting, inserted, startIndex, skipIndex): # join 2 lines
        base = np.copy(starting)
        ins = np.copy(inserted)
        
        for i,note in enumerate(ins):
            base = np.insert(base, (i*skipIndex)+startIndex, [note], axis=0)
            
        return base
        
    def joinLines(self): # join all SATB lines
        #self.notes = np.copy(self.soprano)
        self.notes = self.InsertLine(self.soprano.notes, self.alto.notes, 1, 3)
        self.notes = self.InsertLine(self.notes, self.tenor.notes, 2, 4)
        self.notes = self.InsertLine(self.notes, self.bass.notes, 3, 5)
        
    def OffsetTime(self, maxChange): # adds random time offsets to make output more organic
        for note in self.notes:
            note[2] += random.randint(0,maxChange)
        

#### Voice class
##### Represents individual voices

In [10]:
class Voice: # individual voices
    def __init__(self, allNotes, lowNote, highNote, jump=3, speed=4, time=4096, velocity=64):
        self.range = GetRangeMajor(allNotes,lowNote,highNote) #available ntoes
        self.jump = jump #maximum pitch interval between notes
        self.speed = speed #note length i.e. 4 for quarter, 8 for eighth etc.
        self.time = time #song speed
        self.velocity = velocity #note volume
        self.notes = np.empty([0,4]) #notes output
        self.lowNote = lowNote # lowest note
        self.highNote = highNote # highest note
        self.allNotes = allNotes # all notes in scale
            
        self.duration = self.time / self.speed # time between notes
        
        
    def RandomStartNote(self): # Generate Random first note (for soprano)
        note = random.choice(self.range)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
        
    def RandomJump(self): # Generate Random next note (for soprano)
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        lastIndex = self.range.index(lastNote)
        
        newIndex = -1
        while newIndex < 0 or newIndex >= len(self.range): # stay in range
            newIndex = lastIndex + random.randint(-self.jump,self.jump)
            
        newNote = self.range[newIndex]
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, self.duration, self.duration]]),axis=0)
        
        
    def GenerateLine(self, length): # Generate random line (for soprano)
        self.RandomStartNote()
        
        for n in range(length-1):
            self.RandomJump()
            
            
    def clearNotes(self):
        self.notes = np.empty([0,4])
        
    def GetChordNotes(self, chordNotes): # Get useful notes from all chord notes
        chordNotes = chordNotes[chordNotes >= self.lowNote]
        chordNotes = chordNotes[chordNotes <= self.highNote]
        return chordNotes
    
    def ChooseStartChordNote(self, chordNotes): # Choose Random note in chord
        note = random.choice(chordNotes)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
    def ChooseChordNote(self,chordNotes): # Choose suitable next note in chord
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        
        chordNotes = chordNotes[chordNotes >= lastNote - (self.jump*2)]
        chordNotes = chordNotes[chordNotes <= lastNote + (self.jump*2)]
        newNote = random.choice(chordNotes)
        
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, 0, self.duration]]),axis=0)
        
    def GenerateChordLine(self, chords): # Generate A/T/B lines
        
        firstChord = self.GetChordNotes(chords[0])
        self.ChooseStartChordNote(firstChord)
        
        for c in chords[1:]:
            chord = self.GetChordNotes(c)
            self.ChooseChordNote(chord)

## Model architecture definition

GANs


In [11]:
sample_length = 100
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(16*sample_length, 1)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1) #Binary classification task with 1 output neuron. Could also do with 2.
        self.dropout = nn.Dropout(0.7)
        
    def forward(self, x):
        x = x.view(-1, sample_length*16) # flatten songs...
        out = F.leaky_relu(self.fc1(x), 0.2) #Recall: learky_relu allows some negative values...
#         x = self.dropout(x)
#         x = F.leaky_relu(self.fc2(x), 0.2)
#         x = self.dropout(x)
#         x = F.leaky_relu(self.fc3(x), 0.2)
#         x = self.dropout(x)
#         out = self.fc4(x)
        return out

class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 128) #Encoding size hyperparameter...
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 16*sample_length)
        self.dropout = nn.Dropout(0.3) #Note the use of dropout.

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        out = torch.sigmoid(self.fc4(x)) #Applies sigmoid to output. Ensures all values >= 0
        out = out.reshape(-1,sample_length,16)
        return out

D = Discriminator()
G = Generator()

## Training function

In [12]:
#To help us save the model easier...
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [13]:
import torch.optim as optim

def train(G, D, lr=0.002, batch_size=64, num_epochs=20):



    # optimizers for generator and discriminator
    d_optimizer = optim.Adam(D.parameters(), lr)
    g_optimizer = optim.Adam(G.parameters(), lr)
 
    # define loss function
    criterion = nn.BCEWithLogitsLoss()

    # get the training datasets
    train_data = torch.load(r'D:\engsci\year 3\CLASS\APS360\Project\data\one_hot\Notes_Dataset_test_onehot_ws100.pt')

    # prepare data loader
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # keep track of loss and generated, "fake" samples
    samples = []
    losses = []

    # fixed data for testing
    sample_size=16 #Arbitrary.
    test_noise = np.random.uniform(-1, 1, size=(sample_size, rand_size))
    test_noise = torch.from_numpy(test_noise).float()

    for epoch in range(num_epochs):
        D.train() #To enable dropouts...
        G.train()
        
        for batch_i,real_samples in enumerate(train_loader):
#             print(real_samples.shape)
#             print(real_samples.view(-1, sample_length*16).shape)
            batch_size = batch_size
            real_samples = torch.tensor(real_samples).float()
            
            # === Train the Discriminator ===
            
            d_optimizer.zero_grad()
#Note: calculate losses on real and fake images independently...
            # discriminator losses on real images 
            D_real = D(real_samples)
            labels = torch.ones(len(D_real),1)
#             print('labels.shape: ',labels.shape)
#             print('D_real.shape: ',D_real.shape)
            d_real_loss = criterion(D_real, labels)
            
            # discriminator losses on fake images
            z = np.random.uniform(-1, 1, size=(batch_size, rand_size))
            z = torch.from_numpy(z).float()
            fake_samples = G(z)

            D_fake = D(fake_samples)
            labels = torch.zeros(len(D_fake),1) # fake labels = 0
#             print('labels.shape: ',labels.shape)
#             print('D_fake.shape: ',D_fake.shape)
            d_fake_loss = criterion(D_fake, labels)
            
            # add up losses and update parameters
            d_loss = d_real_loss + d_fake_loss #Add the two losses... Done separately to have the labels easily...
            d_loss.backward() #Backpropagate (computes gradients, right?)
            d_optimizer.step() #Take a step.
            

            # === Train the Generator ===
            g_optimizer.zero_grad()
            
            # generator losses on fake song samples...
            z = np.random.uniform(-1, 1, size=(batch_size, rand_size)) #Initial noise... Why -1 --> 1?? Because that's what tanh is... But pixel range still 0-->1... so...???
            #Can change from uniform to gaussian if you want...
            z = torch.from_numpy(z).float()
            fake_samples = G(z) #Collect fake images generated. Somehow gets 0-->1 range... How...
          
            D_fake = D(fake_samples) 
            labels = torch.ones(len(D_fake),1) #flipped labels
#             print('labels.shape: ',labels.shape)
#             print('D_fake.shape: ',D_fake.shape)
            # compute loss and update parameters
            g_loss = criterion(D_fake, labels)
            g_loss.backward()
            g_optimizer.step()
#Essentially training two models.
        # print loss
        print('Epoch [%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
              % (epoch + 1, num_epochs, d_loss.item(), g_loss.item()))

        # append discriminator loss and generator loss
        losses.append((d_loss.item(), g_loss.item()))
        
        # plot images
        G.eval() #Set to evaluation mode, no longer training... Must implement this for the project if we want dropout to play into it.
        D.eval()
        test_images = G(test_noise)

#         plt.figure(figsize=(9, 3))
#         for k in range(16): #Generates a batch of 16...
#             plt.subplot(2, 8, k+1)
#             plt.imshow(test_images[k,:].data.numpy().reshape(28, 28), cmap='Greys')
#         plt.show()
    
    return losses

In [14]:

rand_size = 100; #arbitrary...

D = Discriminator()
G = Generator()

losses = train(G, D, lr=0.02, batch_size=64, num_epochs=40) #Currently, discriminator is too good...
#Need to build a better Generator architecture... Can start with RNN layer, feed it random noise and then go into some
#fully connected layers?
'''
@Eric Liu:

Potential architecture:
input = random noise of size (batch, number of notes, 16)
RNN --> LSTM or GRU of varying length and varying hidden layers
FCNN(s)
output --> Size of (batch,number of notes,16). Reshape the FCNN output to match this or leave out FCNN and just use GRUs

'''

D:\anacondainstal\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch [1/40], d_loss: 6.2436, g_loss: 0.5225, 
Epoch [2/40], d_loss: 9.8859, g_loss: 0.0032, 
Epoch [3/40], d_loss: 1.6164, g_loss: 2.0687, 
Epoch [4/40], d_loss: 1.0381, g_loss: 4.1645, 
Epoch [5/40], d_loss: 0.7382, g_loss: 3.0895, 
Epoch [6/40], d_loss: 0.2629, g_loss: 2.6413, 
Epoch [7/40], d_loss: 0.0802, g_loss: 3.4489, 
Epoch [8/40], d_loss: 0.0384, g_loss: 4.2438, 
Epoch [9/40], d_loss: 0.0075, g_loss: 5.0493, 
Epoch [10/40], d_loss: 0.0277, g_loss: 5.2659, 
Epoch [11/40], d_loss: 0.0196, g_loss: 5.5687, 
Epoch [12/40], d_loss: 0.0034, g_loss: 6.0232, 
Epoch [13/40], d_loss: 0.0024, g_loss: 6.1818, 
Epoch [14/40], d_loss: 0.0020, g_loss: 6.3157, 
Epoch [15/40], d_loss: 0.0016, g_loss: 6.4579, 
Epoch [16/40], d_loss: 0.0015, g_loss: 6.7055, 
Epoch [17/40], d_loss: 0.0012, g_loss: 6.8233, 
Epoch [18/40], d_loss: 0.0011, g_loss: 6.9452, 
Epoch [19/40], d_loss: 0.0010, g_loss: 7.0420, 
Epoch [20/40], d_loss: 0.0036, g_loss: 7.1042, 
Epoch [21/40], d_loss: 0.0012, g_loss: 6.9015, 
E

'\n@Eric Liu:\n\nPotential architecture:\ninput = random noise of size (batch, number of notes, 16)\nRNN --> LSTM or GRU of varying length and varying hidden layers\nFCNN(s)\noutput --> Size of (batch,number of notes,16). Reshape the FCNN output to match this or leave out FCNN and just use GRUs\n\n'

## Model Testing

Since our model is 'tested' with people listening to it, we need to just generate some samples.
To do so, and because this is a GAN, do as follows:

TO DO: implement a note rounding mechanism to we ensure we always get valid notes...

In [15]:
number_of_samples = 1
test_noise = np.random.uniform(-1, 1, size=(number_of_samples, rand_size))
test_noise = torch.from_numpy(test_noise).float()
test_samples = G(test_noise)
test_samples = test_samples.detach().numpy()
print('test_sample: (Watch for the same notes appearing...)',test_samples)
new_excerpt = np.array(test_samples[0])
print(new_excerpt.shape)
mid = Numpy2Midi(np.array(new_excerpt),'test_from_model_68_GAN')

test_sample: (Watch for the same notes appearing...) [[[0. 0. 1. ... 1. 0. 0.]
  [1. 0. 1. ... 1. 0. 0.]
  [0. 0. 1. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]]
(100, 16)


ValueError: data byte must be in range 0..127